In [20]:
from Plan import CreateSamples_renewed
from Prepare import OT2Commands as ALH
from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib
import matplotlib.pyplot as plt

In [33]:
importlib.reload(CreateSamples_renewed)
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

In [25]:
path = r"Testing Plans\05_7_21_5CB_General_Scan.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [26]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'],
                                                                     unity_filter=True,
                                                                     component_spacing_type='linear')

In [27]:
complete_amounts_added = CreateSamples_renewed.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
stock_dict = CreateSamples_renewed.stock_dictionary(plan['Stock Names'], plan['Stock Concentration Units'], plan['Stock Concentrations'], stock_densities=plan['Stock Densities (g/mL)'])
complete_df = CreateSamples_renewed.calculate_stock_volumes_from_component_concs(plan, complete_amounts_added, stock_dict)
complete_df_solvent_fixed = CreateSamples_renewed.calculate_common_solvent_residual_volumes(complete_df, stock_dict) # be careful with this can subtract more than once

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units


In [47]:
# stock_volumes = CreateSamples_renewed.isolate_common_column(complete_df_solvent_fixed,'stock')
stock_volumes = CreateSamples_renewed.convert_mL_to_uL(complete_df_solvent_fixed)
filtered_stock_volumes = CreateSamples_renewed.filter_general_min_pipette_restriction(stock_volumes, 20)
filtered_stock_volumes = CreateSamples_renewed.filter_general_max_restriction(filtered_stock_volumes, 1000, 'water-stock amount volume uL')
filtered_stock_volumes.reset_index(inplace=True, drop=True)
filtered_stock_volumes[filtered_stock_volumes['5CB concentration wtf']==0]

,5CB concentration wtf,ethanol concentration wtf,water concentration wtf,5CB amount mass g,5CB amount volume uL,ethanol amount mass g,ethanol amount volume uL,water amount mass g,water amount volume uL,5CB-ethanol-stock amount volume uL,ethanol-stock amount volume uL,water-stock amount volume uL
0,0.0,0.100000,0.900000,0.0,0.0,0.100000,126.694539,0.900000,900.000000,0.0,126.742712,900.000000
2,0.0,0.177778,0.822222,0.0,0.0,0.177778,225.234737,0.822222,822.222222,0.0,225.320377,822.222222
5,0.0,0.255556,0.744444,0.0,0.0,0.255556,323.774934,0.744444,744.444444,0.0,323.898043,744.444444
9,0.0,0.333333,0.666667,0.0,0.0,0.333333,422.315132,0.666667,666.666667,0.0,422.475708,666.666667
14,0.0,0.411111,0.588889,0.0,0.0,0.411111,520.855329,0.588889,588.888889,0.0,521.053373,588.888889
19,0.0,0.488889,0.511111,0.0,0.0,0.488889,619.395526,0.511111,511.111111,0.0,619.631038,511.111111
24,0.0,0.566667,0.433333,0.0,0.0,0.566667,717.935724,0.433333,433.333333,0.0,718.208703,433.333333
29,0.0,0.644444,0.355556,0.0,0.0,0.644444,816.475921,0.355556,355.555556,0.0,816.786368,355.555556
34,0.0,0.722222,0.277778,0.0,0.0,0.722222,915.016118,0.277778,277.777778,0.0,915.364033,277.777778
39,0.0,0.800000,0.200000,0.0,0.0,0.800000,1013.556316,0.200000,200.000000,0.0,1013.941698,200.000000


In [48]:
filtered_stock_volumes

,5CB concentration wtf,ethanol concentration wtf,water concentration wtf,5CB amount mass g,5CB amount volume uL,ethanol amount mass g,ethanol amount volume uL,water amount mass g,water amount volume uL,5CB-ethanol-stock amount volume uL,ethanol-stock amount volume uL,water-stock amount volume uL
0,0.00000,0.100000,0.900000,0.00000,0.000000,0.100000,126.694539,0.900000,900.000000,0.0,126.742712,900.000000
1,0.00025,0.100000,0.899750,0.00025,0.247525,0.100000,126.694539,0.899750,899.750000,100.0,26.990237,899.750000
2,0.00000,0.177778,0.822222,0.00000,0.000000,0.177778,225.234737,0.822222,822.222222,0.0,225.320377,822.222222
3,0.00025,0.177778,0.821972,0.00025,0.247525,0.177778,225.234737,0.821972,821.972222,100.0,125.567902,821.972222
4,0.00050,0.177778,0.821722,0.00050,0.495050,0.177778,225.234737,0.821722,821.722222,200.0,25.815427,821.722222
5,0.00000,0.255556,0.744444,0.00000,0.000000,0.255556,323.774934,0.744444,744.444444,0.0,323.898043,744.444444
6,0.00025,0.255556,0.744194,0.00025,0.247525,0.255556,323.774934,0.744194,744.194444,100.0,224.145567,744.194444
7,0.00050,0.255556,0.743944,0.00050,0.495050,0.255556,323.774934,0.743944,743.944444,200.0,124.393092,743.944444
8,0.00075,0.255556,0.743694,0.00075,0.742574,0.255556,323.774934,0.743694,743.694444,300.0,24.640617,743.694444
9,0.00000,0.333333,0.666667,0.00000,0.000000,0.333333,422.315132,0.666667,666.666667,0.0,422.475708,666.666667


In [35]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.3', extra_labware=custom_labware_dict) # make note this must be run constantly ~ if say making loaded dict twice
loaded_dict = ALH.loading_labware(protocol, plan)

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


In [36]:
max_source_vol = 18000 # if the list index reports out of range most likely not enough slots aka too much volume
stock_position_info = ALH.stock_well_ranges(filtered_stock_volumes, loaded_dict, max_source_vol) 
stock_position_info

{'5CB-ethanol-stock amount volume uL': {'Ranges': [[0, 44]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'ethanol-stock amount volume uL': {'Ranges': [[0, 41], [41, 44]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'water-stock amount volume uL': {'Ranges': [[0, 28], [28, 44]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]}}

In [37]:
directions = ALH.create_sample_making_directions(filtered_stock_volumes, stock_position_info, loaded_dict) 
# these directions should be used to make the df, unless there is a final move??

In [316]:
ALH.pipette_volumes_sample_wise(protocol, directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 0.0 from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Returning tip
Dropping tip into A1 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A2 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 126.7427122940431 from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Aspirating 126.7427122940431 uL from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 126.7427122940431 uL into A1 of Fischer 24 Well Plate 7400 ÂµL on 2 at 300.0 uL/sec
Returning tip
Dropping tip into A2 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A3 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 900.0 from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Aspirating 900.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec


In [32]:
filtered_stock_volumes.to_csv('05_07_21_CB_Info', index=False)

In [39]:
final_to_export = CreateSamples_renewed.add_final_location(directions,filtered_stock_volumes, unique_identifier='5CB')

In [45]:
etoh = final_to_export['ethanol concentration wtf'].unique()[4]
final_to_export[final_to_export['ethanol concentration wtf']==etoh]

,UID,Labware,Slot,Well,5CB concentration wtf,ethanol concentration wtf,water concentration wtf,5CB amount mass g,5CB amount volume uL,ethanol amount mass g,ethanol amount volume uL,water amount mass g,water amount volume uL,5CB-ethanol-stock amount volume uL,ethanol-stock amount volume uL,water-stock amount volume uL
14,S2_B7_05-07-2021,Falcon 48 Well Plate 1500 ÂµL,2,B7,0.00000,0.411111,0.588889,0.00000,0.000000,0.411111,520.855329,0.588889,588.888889,0.0,521.053373,588.888889
15,S2_B8_05-07-2021,Falcon 48 Well Plate 1500 ÂµL,2,B8,0.00025,0.411111,0.588639,0.00025,0.247525,0.411111,520.855329,0.588639,588.638889,100.0,421.300898,588.638889
16,S2_C1_05-07-2021,Falcon 48 Well Plate 1500 ÂµL,2,C1,0.00050,0.411111,0.588389,0.00050,0.495050,0.411111,520.855329,0.588389,588.388889,200.0,321.548422,588.388889
17,S2_C2_05-07-2021,Falcon 48 Well Plate 1500 ÂµL,2,C2,0.00075,0.411111,0.588139,0.00075,0.742574,0.411111,520.855329,0.588139,588.138889,300.0,221.795947,588.138889
18,S2_C3_05-07-2021,Falcon 48 Well Plate 1500 ÂµL,2,C3,0.00100,0.411111,0.587889,0.00100,0.990099,0.411111,520.855329,0.587889,587.888889,400.0,122.043472,587.888889
